# MTurk: Approve / Reject Assignments + Post-Task Completion Processing of HITs

In [1]:
import boto3
import json
import pandas as pd
import ast
from simple_colors import *
from typing import Dict
import re
import pprint
import string
from pathlib import Path
import os

In [41]:
from mturk_processing_utils import (
    programmatically_review_HITs,
    get_dataset, 
    get_hit_id_dict, 
    parse_batch, 
    approved_parsed_batch_2_dicts,
    view_assignment,
    mturk,
    get_completetion_progress
)

In [3]:
PROJECT_ROOT_DIR = '/Users/nehasrikanth/Documents/paraphrase-nlu/'
dataset_name = ['atomic', 'snli', 'social']


raw_data_paths = {d: os.path.join(PROJECT_ROOT_DIR, f'raw-data/defeasible-nli/defeasible-{d}') for d in dataset_name}

### Social

In [8]:
social_batches = [
    'mturk_data/creation/social_dnli_annotation_examples_1.json',
]

dnli_social = get_dataset(raw_data_paths['social'], 'social')

Unique premise-hypothesis pairs: 7893 / 77016
Loaded 77016 nonempty train examples...(skipped 3314 examples)
Unique premise-hypothesis pairs: 979 / 9343
Loaded 9343 nonempty dev examples...(skipped 467 examples)
Unique premise-hypothesis pairs: 982 / 9439
Loaded 9439 nonempty test examples...(skipped 421 examples)


In [9]:
_, hit_id_2_example_id_social_1, _ = get_hit_id_dict(social_batches[0])

In [46]:
get_completetion_progress(hit_id_2_example_id_social_1)

Progress
[⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾⌾] 98.7%


In [49]:
view_assignment('3WMINLGALB344QYWTSR6R2QW4F8CAW', dnli_social, hit_id_2_example_id_social_1)


Worker:  A3UP0XSH0HHIQS AssignmentId:  3WMINLGALB344QYWTSR6R2QW4F8CAW
Premise:  
Hypothesis:  Don't brush off complaints as something that isn't serious.
Update:  Many people are complaining about the same thing
Update Type:  strengthener

{'paraphrase_1': 'Many others are expressing their dissatisfaction with the '
                 'same issue.',
 'paraphrase_2': 'Many people are voicing their dissatisfaction with the same '
                 'issue.',
 'paraphrase_3': 'Many people are expressing their displeasure with the same '
                 'issue.'}


In [50]:
response = mturk.approve_assignment(
      AssignmentId='3WMINLGALB344QYWTSR6R2QW4F8CAW',
      RequesterFeedback='Thanks for doing a great job!',
)

In [ ]:
response = mturk.reject_assignment(
    AssignmentId='39LNWE0K4UWFW2YUR74J260LONXUI5',
    RequesterFeedback='These are not 3 paraphrases of the evidence sentence.'
)

In [ ]:
response = mturk.create_worker_block(
    WorkerId='A2B8YUXLFIDKUG',
    Reason='Worker is spamming.'
)